In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
data1 = pd.read_csv('datasets_64642_127042_BankNote.csv')
data1.head()

,variance,skewness,kurtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   variance   1372 non-null   float64
 1   skewness   1372 non-null   float64
 2   kurtosis   1372 non-null   float64
 3   entropy    1372 non-null   float64
 4   class      1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [ ]:
data1["class"].value_counts()

0    762
1    610
Name: class, dtype: int64

In [ ]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["class"]):
    strat_train_set = data1.loc[train_index]
    strat_test_set = data1.loc[test_index]

In [ ]:
 strat_test_set["class"].value_counts()

0    191
1    152
Name: class, dtype: int64

In [ ]:
 strat_train_set["class"].value_counts()

0    571
1    458
Name: class, dtype: int64

In [ ]:
train_set = strat_train_set.drop("class", axis=1) # drop labels for training set
train_labels = strat_train_set["class"].copy()
test_set = strat_test_set.drop("class", axis=1) # drop labels for testing set
test_labels = strat_test_set["class"].copy()

In [ ]:
train_labels

603     0
880     1
107     0
286     0
869     1
       ..
437     0
45      0
1068    1
1080    1
59      0
Name: class, Length: 1029, dtype: int64

In [ ]:
from sklearn.cluster import KMeans
tr_set=train_set.T
kmeans = KMeans(n_clusters=4, random_state=42).fit(tr_set)
tr_fin=kmeans.cluster_centers_.T
te_set=test_set.T
kmeans1 = KMeans(n_clusters=4, random_state=42).fit(te_set)
te_fin=kmeans1.cluster_centers_.T

In [ ]:
tr_fin.shape

(1029, 4)

In [ ]:
train_labels.shape

(1029,)

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
parameters = {'min_samples_leaf':[0.1,1,2,3],'max_depth':[10,50,100], 'criterion':['gini', 'entropy'], 'random_state':[33]}
clf2 = DecisionTreeClassifier()
clf = GridSearchCV(clf2, parameters, scoring = 'balanced_accuracy', cv=10)
res1=clf.fit(tr_fin, train_labels)
res1.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=33)

In [ ]:
res1.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'random_state': 33}

In [ ]:
res1.best_score_

0.9886003051106027

In [ ]:
y_test_pred=res1.best_estimator_.predict(te_fin)

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100

98.83381924198251

In [ ]:
#error rate=100-accuracy
100-(sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100)

1.1661807580174894

In [ ]:
matthews_corrcoef(test_labels, y_test_pred)*100

97.63709010746761

In [ ]:
from sklearn.metrics import precision_score
sklearn.metrics.precision_score(test_labels, y_test_pred, pos_label=1)*100

98.68421052631578

In [ ]:
from sklearn.metrics import recall_score
sklearn.metrics.recall_score(test_labels, y_test_pred, pos_label=1)*100

98.68421052631578

In [ ]:
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(test_labels, y_test_pred, pos_label=1)*100

98.68421052631578